In [1]:
import os
import numpy as np
import pandas as pd
import warnings

from google.colab import drive

warnings.filterwarnings('ignore')
drive.mount("/content/drive")

os.chdir("drive/MyDrive/competition/2022-AI-competition-Round1")
os.listdir()

Mounted at /content/drive


['competition_data',
 'submission',
 'playground.ipynb',
 'AutoML Baseline.ipynb',
 'Evaluator Module.ipynb',
 '2022-08-04 feature EDA.ipynb',
 "2022-08-05 LGB_train(hyeonbin's parameter tuning).ipynb",
 'Optuna Optimization.ipynb',
 '2022-08-06 Regressor model compare.ipynb',
 'catboost_info',
 'evaluator',
 '.git',
 '.gitignore',
 'README.md',
 'Updated CLF.ipynb',
 '2022-08-17 Evaluator Optuna (v0.4).ipynb',
 '2022-08-17 Ensemble Module.ipynb',
 '2022-08-11 Models(XGB_ET) Optimization.ipynb',
 '2022-08-06 Optimize ET.ipynb',
 'feature_EDA_correlation.ipynb',
 'Untitled0.ipynb',
 'model compare.ipynb',
 'GitHub Connection.ipynb',
 '2022-08-24 RF optimization.ipynb']

In [2]:
!pip install -r evaluator/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.2 MB/s 
     |████████████████████████████████| 308 kB 28.1 MB/s 
     |████████████████████████████████| 81 kB 8.8 MB/s 
     |████████████████████████████████| 209 kB 41.4 MB/s 
     |████████████████████████████████| 78 kB 6.1 MB/s 
     |████████████████████████████████| 49 kB 5.3 MB/s 
     |████████████████████████████████| 112 kB 48.2 MB/s 
     |████████████████████████████████| 147 kB 45.8 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=f280e404b069cb7a5854a2d97229594694fa31341f628c8ce46decac74380f1f
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [3]:
from evaluator.evaluator import Evaluator, Model

train_df = pd.read_csv('competition_data/train.csv')
test_df = pd.read_csv("competition_data/test.csv")
submission_df = pd.read_csv("competition_data/sample_submission.csv")

### baseline

In [4]:
Evaluator(
    **Model(train_df, "et", "clf").get_model()
).run()

,accuracy,precision,recall,f1-score,roc_auc,mae,train_acc
fold,,,,,,,
1,0.778993,0.783446,0.837973,0.809793,0.872651,0.302513,NaN
2,0.780869,0.770790,0.850832,0.808836,0.875516,0.305789,NaN
3,0.777118,0.773053,0.848229,0.808898,0.868300,0.311222,NaN
4,0.781807,0.799478,0.829810,0.814362,0.867130,0.306389,NaN
mean,0.779697,0.781692,0.841711,0.810472,0.870899,0.306479,1.0


### optimize

In [10]:
params = {
	'n_estimators': 292, 
	'max_depth': 45, 
	'min_samples_split': 2, 
	'min_samples_leaf': 1, 
	'max_features': 0.729627795893577
}

In [11]:
Evaluator(
    **Model(train_df, "et", "clf", **params).get_model()
).run()

,accuracy,precision,recall,f1-score,roc_auc,mae,train_acc
fold,,,,,,,
1,0.782432,0.794748,0.825724,0.809940,0.870618,0.293912,NaN
2,0.779306,0.776829,0.834768,0.804757,0.876743,0.295358,NaN
3,0.776180,0.780179,0.831928,0.805223,0.868113,0.301992,NaN
4,0.783995,0.805938,0.823848,0.814795,0.867077,0.297001,NaN
mean,0.780478,0.789424,0.829067,0.808679,0.870638,0.297066,1.0


In [5]:
model = Model(train_df, "et", "clf")

In [12]:
initial_params = (
    ("n_estimators", "int", (200, 500)),
    ("max_depth", "int", (10, 70)),
    ("min_samples_split", "int", (2, 5)),
    ("max_features", "float", (0.6, 0.8)),
    ("max_leaf_nodes", "int", (200, 1000)),
)
model.optimize(initial_params, n_runs=2)

[I 2022-08-24 06:27:50,602] A new study created in memory with name: no-name-15d60fde-5b85-4e74-87a7-7b71401a7956
[I 2022-08-24 06:29:20,615] Trial 0 finished with value: 0.8547220835588512 and parameters: {'n_estimators': 355, 'max_depth': 44, 'min_samples_split': 4, 'max_features': 0.6352298640193775, 'max_leaf_nodes': 764}. Best is trial 0 with value: 0.8547220835588512.
[I 2022-08-24 06:31:46,451] Trial 1 finished with value: 0.8535512551973433 and parameters: {'n_estimators': 493, 'max_depth': 61, 'min_samples_split': 5, 'max_features': 0.7693449517030381, 'max_leaf_nodes': 738}. Best is trial 0 with value: 0.8547220835588512.


KeyboardInterrupt: ignored

In [ ]:
evaluator = Evaluator(
    **model.get_model()
)
evaluator.run()